<a href="https://colab.research.google.com/github/tejas2008/FakeAccountDetection/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)
from nltk.stem import PorterStemmer 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
from nltk.corpus import stopwords 
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

In [4]:
len(positive_tweets)

5000

In [17]:

test_pos = positive_tweets[4000:]
train_pos = positive_tweets[:4000]
test_neg = negative_tweets[4000:]
train_neg = negative_tweets[:4000]
train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [18]:
len(train_x)

8000

In [19]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [20]:
train_y

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [21]:
stemmer = PorterStemmer()
def preprocess(tweet):
  punc = '''!()-[]{};:'"\, <>./?@#$%^&*'''
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
  tweet = re.sub(r'#', '', tweet)
  tokn_word = tknzr.tokenize(tweet)
  stp_words = set(stopwords.words('english')) 
  out = []
  for i in tokn_word:
    if i not in stp_words and i not in punc:
      j = stemmer.stem(i)
      out.append(j)
  return out


In [22]:
for i in range(len(train_x)):
  train_x[i] = preprocess(train_x[i])
print(train_x[0])

['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [25]:
fre = {}
def frequency(l):
  freqs = {}
  for i in range(4000):
    for word in l[i]:
      if (word,1) in freqs:
        freqs[(word,1)] += 1
      else:
        freqs[(word,1)] = 1 
  for i in range(4000,8000):
    for word in l[i]:
      if (word,0) in freqs:
        freqs[(word,0)] += 1
      else:
        freqs[(word,0)] = 1 
  return freqs
fre = frequency(train_x)
print(fre)

{('followfriday', 1): 23, ('top', 1): 30, ('engag', 1): 7, ('member', 1): 14, ('commun', 1): 27, ('week', 1): 72, (':)', 1): 2847, ('hey', 1): 60, ('jame', 1): 7, ('how', 1): 18, ('odd', 1): 2, (':/', 1): 5, ('pleas', 1): 80, ('call', 1): 27, ('contact', 1): 4, ('centr', 1): 1, ('02392441234', 1): 1, ('abl', 1): 6, ('assist', 1): 1, ('mani', 1): 28, ('thank', 1): 504, ('listen', 1): 14, ('last', 1): 39, ('night', 1): 55, ('As', 1): 3, ('you', 1): 95, ('bleed', 1): 2, ('amaz', 1): 41, ('track', 1): 5, ('when', 1): 10, ('scotland', 1): 2, ('congrat', 1): 15, ('yeaaaah', 1): 1, ('yippppi', 1): 1, ('accnt', 1): 2, ('verifi', 1): 2, ('rqst', 1): 1, ('succeed', 1): 1, ('got', 1): 57, ('blue', 1): 8, ('tick', 1): 1, ('mark', 1): 1, ('fb', 1): 1, ('profil', 1): 2, ('15', 1): 4, ('day', 1): 187, ('thi', 1): 31, ('one', 1): 90, ('irresist', 1): 2, ('flipkartfashionfriday', 1): 16, ('We', 1): 65, ('like', 1): 187, ('keep', 1): 55, ('love', 1): 336, ('custom', 1): 4, ('wait', 1): 55, ('long', 1): 

In [ ]:
def vec_for_model(fre,l):
  